# 결측치, 이상치 처리 
## Version 4

In [1]:
!uv pip install sentence-transformers scikit-learn tqdm --break-system-packages

Using Python 3.10.13 environment at: /data/ephemeral/home/sojin/.venv
Audited 3 packages in 6ms


In [2]:
import pandas as pd
import numpy as np
import regex
from typing import Tuple, Dict
import warnings
import os
from tqdm import tqdm

warnings.filterwarnings("ignore")

In [3]:
# 필요한 경우에만 import (Sentence-BERT는 무거우므로)
try:
    from sentence_transformers import SentenceTransformer
    from sklearn.cluster import AgglomerativeClustering
    from sklearn.metrics.pairwise import cosine_similarity

    SBERT_AVAILABLE = True
except ImportError:
    SBERT_AVAILABLE = False
    print("{WARNING!} sentence-transformers 또는 sklearn이 설치되지 않았습니다.")
    print("Step 4(클러스터링)을 건너뜁니다.")


class AdvancedCategoryHandler:
    """고급 카테고리 결측치 처리"""

    def __init__(self, use_clustering: bool = True):
        self.use_clustering = use_clustering and SBERT_AVAILABLE
        self.category_keywords = self._build_category_keywords()
        self.sbert_model = None

    def _build_category_keywords(self):
        """카테고리별 키워드 사전"""
        return {
            "Fiction": [
                "novel",
                "story",
                "tales",
                "fiction",
                "romance",
                "mystery",
                "thriller",
                "fantasy",
                "adventure",
            ],
            "History": [
                "history",
                "historical",
                "war",
                "ancient",
                "medieval",
                "century",
                "civilization",
            ],
            "Biography": [
                "biography",
                "autobiography",
                "memoir",
                "life of",
                "story of",
            ],
            "Science": [
                "science",
                "physics",
                "chemistry",
                "biology",
                "astronomy",
                "scientific",
            ],
            "Technology": [
                "computer",
                "programming",
                "software",
                "technology",
                "digital",
                "internet",
                "web",
                "python",
                "java",
                "code",
            ],
            "Business": [
                "business",
                "management",
                "marketing",
                "finance",
                "economics",
                "entrepreneur",
            ],
            "Self-Help": [
                "self-help",
                "self help",
                "motivation",
                "success",
                "improvement",
                "guide to",
            ],
            "Children": ["children", "kids", "young", "picture book", "illustrated"],
            "Cooking": ["cook", "recipe", "food", "kitchen", "culinary", "cuisine"],
            "Art": [
                "art",
                "artist",
                "painting",
                "design",
                "illustration",
                "photography",
            ],
            "Religion": [
                "bible",
                "god",
                "christian",
                "islam",
                "buddhist",
                "prayer",
                "spiritual",
            ],
            "Poetry": ["poem", "poetry", "verse"],
            "Travel": ["travel", "guide", "journey", "destination", "tourist"],
            "Health": ["health", "medical", "medicine", "fitness", "diet", "wellness"],
            "Education": ["education", "teaching", "learning", "textbook", "study"],
        }

    def fill_category_missing(self, books: pd.DataFrame) -> Dict:
        """다단계 결측치 처리"""
        books = books.copy()
        original_missing = books["category"].isna().sum()

        print(f"\n{'='*10}")
        print(f"> Advanced Category 결측치 처리 시작")
        print(f"{'='*10}")
        print(
            f"원본 결측치: {original_missing:,}개 ({original_missing/len(books)*100:.2f}%)\n"
        )

        report = {"original_missing": original_missing}

        # Step 1: 동일 저자 + 출판연도 ±2년
        books, step1_filled = self._fill_by_author_year(books)
        report["step1_filled"] = step1_filled

        # Step 2: 동일 Publisher
        books, step2_filled = self._fill_by_publisher(books)
        report["step2_filled"] = step2_filled

        # Step 3: Book Title 키워드
        books, step3_filled = self._fill_by_title_keywords(books)
        report["step3_filled"] = step3_filled

        # Step 4: Sentence-BERT 클러스터링
        if self.use_clustering:
            books, step4_filled = self._fill_by_clustering(books)
            report["step4_filled"] = step4_filled
        else:
            step4_filled = 0
            report["step4_filled"] = 0

        # Step 5: unknown
        books["category"].fillna("unknown", inplace=True)
        final_unknown = (books["category"] == "unknown").sum()
        report["final_unknown"] = final_unknown

        print(f"{'='*10}")
        print(f"> Category 결측치 처리 완료")
        print(f"{'='*10}")

        print(f"Step 1 (저자+연도): {step1_filled:,}개")
        print(f"Step 2 (Publisher): {step2_filled:,}개")
        print(f"Step 3 (키워드): {step3_filled:,}개")
        print(f"Step 4 (클러스터링): {step4_filled:,}개")
        print(f"Step 5 (unknown): {final_unknown:,}개")
        print(
            f"총 해결: {original_missing - final_unknown:,}개 ({(original_missing - final_unknown)/original_missing*100:.2f}%)\n"
        )

        return books, report

    def _fill_by_author_year(self, books: pd.DataFrame) -> Tuple[pd.DataFrame, int]:
        # Step 1: 동일 저자 + 출판연도 ±2년
        print("> Step 1: 동일 저자 + 출판연도 ±2년 이내")
        books = books.copy()
        missing_mask = books["category"].isna()
        filled_count = 0

        for idx in tqdm(books[missing_mask].index, desc="  처리 중"):
            row = books.loc[idx]
            if pd.isna(row["book_author"]) or row["book_author"] == "unknown":
                continue

            same_author = books[
                (books["book_author"] == row["book_author"])
                & (
                    books["year_of_publication"].between(
                        row["year_of_publication"] - 2, row["year_of_publication"] + 2
                    )
                )
                & (books["category"].notna())
            ]

            if len(same_author) > 0:
                mode_category = same_author["category"].mode()
                if len(mode_category) > 0:
                    books.loc[idx, "category"] = mode_category[0]
                    filled_count += 1

        remaining = books["category"].isna().sum()
        print(f"  ✓ {filled_count:,}개 해결 | 남은 결측치: {remaining:,}개\n")
        return books, filled_count

    def _fill_by_publisher(self, books: pd.DataFrame) -> Tuple[pd.DataFrame, int]:
        # Step 2: 동일 Publisher
        print("> Step 2: 동일 Publisher")
        books = books.copy()
        missing_mask = books["category"].isna()
        filled_count = 0

        # Publisher별 최빈 카테고리 계산
        publisher_category_map = {}
        for publisher in books[missing_mask]["publisher"].unique():
            if pd.isna(publisher):
                continue
            same_publisher = books[
                (books["publisher"] == publisher) & (books["category"].notna())
            ]
            if len(same_publisher) > 0:
                mode_category = same_publisher["category"].mode()
                if len(mode_category) > 0:
                    publisher_category_map[publisher] = mode_category[0]

        # 적용
        for idx in books[missing_mask].index:
            publisher = books.loc[idx, "publisher"]
            if not pd.isna(publisher) and publisher in publisher_category_map:
                books.loc[idx, "category"] = publisher_category_map[publisher]
                filled_count += 1

        remaining = books["category"].isna().sum()
        print(f"  ✓ {filled_count:,}개 해결 | 남은 결측치: {remaining:,}개\n")
        return books, filled_count

    def _fill_by_title_keywords(self, books: pd.DataFrame) -> Tuple[pd.DataFrame, int]:
        # Step 3: Title 키워드 매칭
        print("> Step 3: Book Title 키워드 매칭")
        books = books.copy()
        missing_mask = books["category"].isna()
        filled_count = 0

        for idx in tqdm(books[missing_mask].index, desc="  처리 중"):
            title = books.loc[idx, "book_title"]
            if pd.isna(title):
                continue

            title_lower = str(title).lower()
            for category, keywords in self.category_keywords.items():
                if any(keyword in title_lower for keyword in keywords):
                    books.loc[idx, "category"] = category
                    filled_count += 1
                    break

        remaining = books["category"].isna().sum()
        print(f"  ✓ {filled_count:,}개 해결 | 남은 결측치: {remaining:,}개\n")
        return books, filled_count

    def _fill_by_clustering(
        self, books: pd.DataFrame, n_clusters: int = 650
    ) -> Tuple[pd.DataFrame, int]:

        # Step 4: Sentence-BERT 클러스터링
        print(f"> Step 4: Sentence-BERT 클러스터링 (목표: {n_clusters}개)")
        books = books.copy()
        filled_count = 0

        if self.sbert_model is None:
            print("** 모델 로딩 시작")
            self.sbert_model = SentenceTransformer("all-MiniLM-L6-v2")

        books_with_category = books[books["category"].notna()].copy()
        unique_categories = books_with_category["category"].unique()

        if len(unique_categories) < n_clusters:
            print(f"**** 카테고리 수({len(unique_categories)})가 목표보다 적어 스킵\n")
            return books, 0

        print(f"** 임베딩 생성 시작")
        category_embeddings = self.sbert_model.encode(
            unique_categories.tolist(), show_progress_bar=False, batch_size=32
        )

        print(f"** 클러스터링 시작")
        clustering = AgglomerativeClustering(
            n_clusters=n_clusters, metric="cosine", linkage="average"
        )
        cluster_labels = clustering.fit_predict(category_embeddings)

        category_to_cluster = dict(zip(unique_categories, cluster_labels))

        # 클러스터 대표 카테고리
        cluster_representatives = {}
        for cluster_id in range(n_clusters):
            categories_in_cluster = [
                cat for cat, cid in category_to_cluster.items() if cid == cluster_id
            ]
            if categories_in_cluster:
                category_counts = books_with_category[
                    books_with_category["category"].isin(categories_in_cluster)
                ]["category"].value_counts()
                cluster_representatives[cluster_id] = category_counts.index[0]

        # 결측치 처리
        missing_mask = books["category"].isna()
        missing_books = books[missing_mask].copy()

        if len(missing_books) > 0:
            print(f"** 결측치 {len(missing_books):,}개 처리 시작")
            missing_titles = missing_books["book_title"].fillna("").tolist()
            missing_embeddings = self.sbert_model.encode(
                missing_titles, show_progress_bar=False, batch_size=32
            )

            # 클러스터 중심
            cluster_centers = {}
            for cluster_id in range(n_clusters):
                categories_in_cluster = [
                    cat for cat, cid in category_to_cluster.items() if cid == cluster_id
                ]
                if categories_in_cluster:
                    cluster_indices = [
                        i
                        for i, cat in enumerate(unique_categories)
                        if cat in categories_in_cluster
                    ]
                    cluster_centers[cluster_id] = category_embeddings[
                        cluster_indices
                    ].mean(axis=0)

            # 매칭
            for i, (idx, row) in enumerate(missing_books.iterrows()):
                if pd.isna(row["book_title"]) or str(row["book_title"]).strip() == "":
                    continue

                similarities = {
                    cluster_id: cosine_similarity([missing_embeddings[i]], [center])[0][
                        0
                    ]
                    for cluster_id, center in cluster_centers.items()
                }

                best_cluster = max(similarities, key=similarities.get)
                if best_cluster in cluster_representatives:
                    books.loc[idx, "category"] = cluster_representatives[best_cluster]
                    filled_count += 1

        remaining = books["category"].isna().sum()
        print(f"  ✓ {filled_count:,}개 해결 | 남은 결측치: {remaining:,}개\n")
        return books, filled_count

In [4]:
# Language를 ISBN 기반으로 추론하는 클래스
class LanguageParserByISBN:
    def __init__(self):
        """ISBN 접두사 → Language 매핑"""
        self.isbn_language_map = {
            # 영어권 (0, 1)
            "0": "en",
            "1": "en",
            # 프랑스어권 (2)
            "2": "fr",
            # 독일어권 (3)
            "3": "de",
            # 일본 (4)
            "4": "ja",
            # 러시아 (5)
            "5": "ru",
            # 중국 (7)
            "7": "zh",
            # 체코/슬로바키아 (80)
            "80": "cs",
            # 인도 (81)
            "81": "hi",
            # 노르웨이 (82)
            "82": "no",
            # 폴란드 (83)
            "83": "pl",
            # 스페인 (84)
            "84": "es",
            # 브라질 (85)
            "85": "pt",
            # 유고슬라비아 (86)
            "86": "sr",
            # 덴마크 (87)
            "87": "da",
            # 이탈리아 (88)
            "88": "it",
            # 한국 (89)
            "89": "ko",
            # 네덜란드 (90)
            "90": "nl",
            # 스웨덴 (91)
            "91": "sv",
            # 포르투갈 (972)
            "972": "pt",
            # 터키 (975)
            "975": "tr",
            # ISBN-13 변환 접두사
            "978": "en",
            "979": "en",
        }

    def extract_isbn_prefix(self, isbn: str) -> str:
        """ISBN에서 Group Identifier 추출"""
        if pd.isna(isbn):
            return None

        isbn_str = str(isbn).strip()

        # ISBN-13인 경우 (978 또는 979로 시작)
        if isbn_str.startswith(("978", "979")):
            if len(isbn_str) >= 4:
                return isbn_str[3]

        # 3자리 접두사 확인 (예: 972)
        if len(isbn_str) >= 3 and isbn_str[:3] in self.isbn_language_map:
            return isbn_str[:3]

        # 2자리 접두사 확인 (예: 80, 84)
        if len(isbn_str) >= 2 and isbn_str[:2] in self.isbn_language_map:
            return isbn_str[:2]

        # 1자리 접두사 (예: 0, 1, 2, 3)
        if len(isbn_str) >= 1 and isbn_str[0] in self.isbn_language_map:
            return isbn_str[0]

        return None

    def infer_language_from_isbn(self, isbn: str, default: str = "en") -> str:
        """ISBN으로부터 Language 추론"""
        prefix = self.extract_isbn_prefix(isbn)

        if prefix is None:
            return default

        return self.isbn_language_map.get(prefix, default)

In [ ]:
class DataCleaningPipeline:

    def __init__(
        self,
        data_path: str,
        output_path: str = None,
        use_advanced_category: bool = True,
    ):
        self.data_path = data_path
        self.output_path = output_path or data_path
        self.cleaning_report = {}
        self.use_advanced_category = use_advanced_category
        self.isbn_parser = LanguageParserByISBN()
        self.category_handler = (
            AdvancedCategoryHandler() if use_advanced_category else None
        )

    def load_data(
        self,
    ) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:

        print("> 원본 데이터 로딩 시작")
        users = pd.read_csv(self.data_path + "users.csv")
        books = pd.read_csv(
            "/data/ephemeral/home/sojin/data/books_with_categories.csv"
        )  # TODO: 경로수정
        train = pd.read_csv(self.data_path + "train_ratings.csv")
        test = pd.read_csv(self.data_path + "test_ratings.csv")

        self.cleaning_report["original_sizes"] = {
            "users": len(users),
            "books": len(books),
            "train": len(train),
            "test": len(test),
        }

        print(f"- Users: {len(users):,} rows")
        print(f"- Books: {len(books):,} rows")
        print(f"- Train: {len(train):,} rows")
        print(f"- Test: {len(test):,} rows\n")

        return users, books, train, test

    # 나이 데이터 정제
    def clean_age(self, users: pd.DataFrame) -> pd.DataFrame:
        """
        결측치 -> 중간값으로 대체 (평균은 이상치에 민감하니까)
        """
        print("> User Age 정제 시작\n")

        users = users.copy()
        original_missing = users["age"].isnull().sum()

        # 중간값 계산
        reasonable_age = users["age"][(users["age"] >= 5) & (users["age"] <= 100)]
        median_age = reasonable_age.median()

        # 결측치 → 중앙값
        users["age"].fillna(median_age, inplace=True)
        print(f"  - 결측치 {original_missing:,}건 → 중앙값({median_age:.0f}세)로 대체")

        self.cleaning_report["age"] = {
            "original_missing": original_missing,
            "median_used": median_age,
        }

        print(f"> Age 정제 완료 (중앙값: {median_age:.0f}세)\n")
        return users

    # 위치 데이터 정제
    def clean_location(self, users: pd.DataFrame) -> pd.DataFrame:
        """
        1. 'n/a, n/a, n/a' → NaN으로 변환
        2. Country/State/City 파싱
        3. 결측치 보완 (State 있으면 Country 추론)
        """
        print("> User Location 정제 시작")

        users = users.copy()

        def split_location(x: str) -> list:
            res = x.split(",")
            res = [i.strip().lower() for i in res]
            res = [regex.sub(r"[^a-zA-Z/ ]", "", i) for i in res]
            res = [i if i not in ["n/a", ""] else np.nan for i in res]
            res.reverse()

            for i in range(len(res) - 1, 0, -1):
                if (res[i] in res[:i]) and (not pd.isna(res[i])):
                    res.pop(i)
            return res

        # Location 파싱
        users["location_list"] = users["location"].apply(split_location)
        users["location_country"] = users["location_list"].apply(
            lambda x: x[0] if len(x) > 0 else np.nan
        )
        users["location_state"] = users["location_list"].apply(
            lambda x: x[1] if len(x) > 1 else np.nan
        )
        users["location_city"] = users["location_list"].apply(
            lambda x: x[2] if len(x) > 2 else np.nan
        )

        # 결측치 보완
        filled_countries = 0
        filled_states = 0

        for idx, row in users.iterrows():
            # State는 있는데 Country가 없는 경우
            if (not pd.isna(row["location_state"])) and pd.isna(
                row["location_country"]
            ):
                fill_country = users[users["location_state"] == row["location_state"]][
                    "location_country"
                ].mode()
                if len(fill_country) > 0:
                    users.loc[idx, "location_country"] = fill_country[0]
                    filled_countries += 1

            # City는 있는데 State가 없는 경우
            elif (not pd.isna(row["location_city"])) and pd.isna(row["location_state"]):
                if not pd.isna(row["location_country"]):
                    fill_state = users[
                        (users["location_country"] == row["location_country"])
                        & (users["location_city"] == row["location_city"])
                    ]["location_state"].mode()
                    if len(fill_state) > 0:
                        users.loc[idx, "location_state"] = fill_state[0]
                        filled_states += 1

        # 남은건 최빈값으로 대체
        mode_country = (
            users["location_country"].mode()[0]
            if not users["location_country"].mode().empty
            else "unknown"
        )
        original_country_missing = users["location_country"].isnull().sum()
        users["location_country"].fillna(mode_country, inplace=True)

        print(f"** Country: {filled_countries}건 복원")
        print(
            f"** Country: 최빈값({mode_country})으로 {original_country_missing - filled_countries}건 대체"
        )
        print(f"** State: {filled_states}건 복원")

        users = users.drop(["location", "location_list"], axis=1)

        self.cleaning_report["location"] = {
            "filled_countries": filled_countries,
            "filled_states": filled_states,
            "mode_country": mode_country,
        }

        print(f"> Location 정제 완료\n")
        return users

    # 책 데이터 정제
    def clean_books(self, books: pd.DataFrame) -> pd.DataFrame:
        """
        1. year_of_publication: 1900 미만 또는 2025 초과 → 중간값으로 대체
        2. language: 결측치 → ISBN 번호 기반 추론 → 추론 실패 시 최빈값으로 대체
        3. category: 결측치 →
            1. 동일저자 && 출판년도 오차2 내 => 같은 카테고리로
            2. 그 출판사의 최빈 카테고리로
            3. 제목 사전 만들어서 키워드 맵핑
            4. Sentence Bert 임베딩으로 카테고리 통합
            5. 나머지는 unknown


        4. book_author: 결측치 → 'unknown'
        """
        print("> Books 데이터 정제 시작")

        books = books.copy()

        # 1. Year of Publication 결측치 → 중간값으로
        reasonable_years = books["year_of_publication"][
            (books["year_of_publication"] >= 1900)
            & (books["year_of_publication"] <= 2025)
        ]
        median_year = reasonable_years.median()

        outlier_year_count = (
            (books["year_of_publication"] < 1900)
            | (books["year_of_publication"] > 2025)
        ).sum()

        books.loc[
            (books["year_of_publication"] < 1900)
            | (books["year_of_publication"] > 2025),
            "year_of_publication",
        ] = median_year

        print(
            f"** 출판연도: 비정상 범위 {outlier_year_count}건 → 중간값({median_year:.0f})으로 대체"
        )

        # 2. Language 결측치 → ISBN 번호 기반 추론 → 추론 실패 시 최빈값으로 대체
        original_missing = books["language"].isnull().sum()
        print(f"** 원본 결측치: {original_missing:,}건")

        missing_mask = books["language"].isnull()
        books.loc[missing_mask, "language"] = books.loc[missing_mask, "isbn"].apply(
            self.isbn_parser.infer_language_from_isbn
        )

        # ISBN 추론 후 남은 결측치
        remaining_missing = books["language"].isnull().sum()
        isbn_filled = original_missing - remaining_missing

        print(
            f"** ISBN으로 유추한 개수: {isbn_filled:,}건 ({isbn_filled/original_missing*100:.1f}%)"
        )

        # 남은 결측치는 최빈값으로 대체
        if remaining_missing > 0:
            mode_language = (
                books["language"].mode()[0]
                if not books["language"].mode().empty
                else "en"
            )
            books["language"].fillna(mode_language, inplace=True)
            print(f"** 최빈값({mode_language})으로 나머지 {remaining_missing:,}건 대체")

        # ISBN으로 추론한 언어 분포 확인
        if isbn_filled > 0:
            filled_langs = books.loc[missing_mask, "language"].value_counts().head(5)
            print(f"\n  [ISBN으로 추론한 언어 분포 확인 (Top 5)]")
            for lang, count in filled_langs.items():
                print(f"- {lang}: {count:,}건")

        # 3. Category 정제
        def str2list(x):
            if pd.isna(x):
                return np.nan
            if x[0] != "[":
                return x.split(", ")
            return x[1:-1].split(", ")

        books["category"] = books["category"].apply(
            lambda x: str2list(x)[0] if not pd.isna(x) else np.nan
        )

        if self.use_advanced_category:
            books, category_report = self.category_handler.fill_category_missing(books)
            self.cleaning_report["category_advanced"] = category_report
            category_missing = category_report.get("final_unknown", 0)

        else:
            books["category"].fillna("unknown", inplace=True)
            category_missing = (books["category"] == "unknown").sum()
            print(f"** Category: 결측치 → 'unknown'으로 대체")

        # 4. Book Author 결측치 → unknown
        author_missing = books["book_author"].isnull().sum()
        books["book_author"].fillna("unknown", inplace=True)
        print(f"** Book Author: 결측치 {author_missing}건 → 'unknown'으로 대체")

        self.cleaning_report["books"] = {
            "outlier_year_count": outlier_year_count,
            "median_year": median_year,
            "language_missing": original_missing,
            "category_missing": category_missing,
            "author_missing": author_missing,
        }

        print(f"> Books 정제 완료\n")
        return books

    # 평점 데이터 정제
    def clean_ratings(
        self, train: pd.DataFrame, test: pd.DataFrame
    ) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """
        중복 데이터 제거
        """
        print("> Ratings 데이터 정제 시작")

        train = train.copy()
        test = test.copy()

        # 중복 제거 (user_id + isbn 기준)
        train_duplicates = train.duplicated(subset=["user_id", "isbn"]).sum()
        if train_duplicates > 0:
            train = train.drop_duplicates(subset=["user_id", "isbn"], keep="last")
            print(f"** Train: 중복 데이터 {train_duplicates}건 제거")

        test_duplicates = test.duplicated(subset=["user_id", "isbn"]).sum()
        if test_duplicates > 0:
            test = test.drop_duplicates(subset=["user_id", "isbn"], keep="last")
            print(f"** Test: 중복 데이터 {test_duplicates}건 제거")

        self.cleaning_report["ratings"] = {
            "train_duplicates": train_duplicates,
            "test_duplicates": test_duplicates,
        }

        print(f"> Ratings 정제 완료\n")
        return train, test

    def save_cleaned_data(
        self,
        users: pd.DataFrame,
        books: pd.DataFrame,
        train: pd.DataFrame,
        test: pd.DataFrame,
    ):
        print(">>> 정제된 데이터 저장 시작")

        additional_path = (
            self.output_path + "v4/"
        )  # TODO: 데이터 버전별로 파일명 수정하기
        os.makedirs(additional_path, exist_ok=True)
        print(f"** 저장 경로: {additional_path}")

        # 저장 시, Location 컬럼 재결합
        if all(
            col in users.columns
            for col in ["location_country", "location_state", "location_city"]
        ):

            def combine_location(row):
                parts = []

                # city, state, country
                if pd.notna(row["location_city"]) and row["location_city"] != "unknown":
                    parts.append(str(row["location_city"]))

                if (
                    pd.notna(row["location_state"])
                    and row["location_state"] != "unknown"
                ):
                    parts.append(str(row["location_state"]))

                if (
                    pd.notna(row["location_country"])
                    and row["location_country"] != "unknown"
                ):
                    parts.append(str(row["location_country"]))

                # 비어있으면 기본값
                if not parts:
                    return "unknown, unknown, unknown"

                return ", ".join(parts)

            users["location"] = users.apply(combine_location, axis=1)

            # 파싱된 컬럼 제거
            users = users.drop(
                ["location_country", "location_state", "location_city"], axis=1
            )

        users.to_csv(additional_path + "users.csv", index=False)
        books.to_csv(additional_path + "books.csv", index=False)
        train.to_csv(additional_path + "train_ratings.csv", index=False)
        test.to_csv(additional_path + "test_ratings.csv", index=False)

        print(f"  - {additional_path}users.csv")
        print(f"  - {additional_path}books.csv")
        print(f"  - {additional_path}train_ratings.csv")
        print(f"  - {additional_path}test_ratings.csv")

        print("\n<<< 저장 완료! >>>\n")

    def generate_cleaning_report(self):
        print("=" * 10)
        print(" " * 20 + ">>> 데이터 정제 결과 >>> ")
        print("=" * 10)

        print("\n[원본 데이터 크기]")
        for key, value in self.cleaning_report["original_sizes"].items():
            print(f"  - {key.capitalize()}: {value:,} rows")

        print("\n[Age 정제]")
        age_report = self.cleaning_report.get("age", {})
        print(f"- 결측치 대체: {age_report.get('original_missing', 0):,}건")

        print("\n[Location 정제]")
        loc_report = self.cleaning_report.get("location", {})
        print(f"- Country 복원: {loc_report.get('filled_countries', 0)}건")
        print(f"- State 복원: {loc_report.get('filled_states', 0)}건")
        print(f"- Country 최빈값: {loc_report.get('mode_country', 'N/A')}")

        print("\n[Books 정제]")
        books_report = self.cleaning_report.get("books", {})
        print(f"- 출판연도 이상치: {books_report.get('outlier_year_count', 0)}건")
        print(f"- 사용된 중앙 연도: {books_report.get('median_year', 0):.0f}")
        print(f"- Language 결측치: {books_report.get('language_missing', 0):,}건")
        print(f"- Category 결측치: {books_report.get('category_missing', 0):,}건")
        print(f"- Author 결측치: {books_report.get('author_missing', 0)}건")
        if "category_advanced" in self.cleaning_report:
            cat_adv = self.cleaning_report["category_advanced"]
            print("\n[Advanced Category 처리 상세]")
            print(f"- 원본 결측치: {cat_adv.get('original_missing', 0):,}건")
            print(f"- Step 1 (저자+연도): {cat_adv.get('step1_filled', 0):,}건")
            print(f"- Step 2 (Publisher): {cat_adv.get('step2_filled', 0):,}건")
            print(f"- Step 3 (키워드): {cat_adv.get('step3_filled', 0):,}건")
            print(f"- Step 4 (클러스터링): {cat_adv.get('step4_filled', 0):,}건")
            print(f"- 최종 unknown: {cat_adv.get('final_unknown', 0):,}건")

        print("\n[Ratings 정제]")
        ratings_report = self.cleaning_report.get("ratings", {})
        print(f"- Train 중복 제거: {ratings_report.get('train_duplicates', 0)}건")
        print(f"- Test 중복 제거: {ratings_report.get('test_duplicates', 0)}건")

        print("\n" + "=" * 10)

    def run(self):
        """전체 실행"""

        # 1. 데이터 로드
        users, books, train, test = self.load_data()

        # 2. Users 정제
        users = self.clean_age(users)
        users = self.clean_location(users)

        # 3. Books 정제
        books = self.clean_books(books)

        # 4. Ratings 정제
        train, test = self.clean_ratings(train, test)

        # 5. 저장
        self.save_cleaned_data(users, books, train, test)

        # 6. 보고서 출력
        self.generate_cleaning_report()

        return users, books, train, test


# 사용 예시
if __name__ == "__main__":
    DATA_PATH = "/data/ephemeral/home/sojin/data/"  # TODO: 원본 데이터 경로 설정

    # 파이프라인 실행
    pipeline = DataCleaningPipeline(DATA_PATH)
    users_cleaned, books_cleaned, train_cleaned, test_cleaned = pipeline.run()

    print("\n🎉 🎉 🎉 모든 작업 완료!")

> 원본 데이터 로딩 중...
  - Users: 68,092 rows
  - Books: 149,570 rows
  - Train: 306,795 rows
  - Test: 76,699 rows

> User Age 정제 시작

  - 결측치 27,833건 → 중앙값(34세)로 대체
> Age 정제 완료 (중앙값: 34세)

> User Location 정제 시작
** Country: 586건 복원
** Country: 최빈값(usa)으로 932건 대체
** State: 1140건 복원
> Location 정제 완료

> Books 데이터 정제 시작
** 출판연도: 비정상 범위 3건 → 중간값(1996)으로 대체
** 원본 결측치: 67,227건
** ISBN으로 유추한 개수: 67,227건 (100.0%)

  [ISBN으로 추론한 언어 분포 확인 (Top 5)]
- en: 56,369건
- de: 5,284건
- fr: 2,420건
- es: 2,415건
- it: 355건

> Advanced Category 결측치 처리 시작
원본 결측치: 0개 (0.00%)

> Step 1: 동일 저자 + 출판연도 ±2년 이내


  처리 중: 0it [00:00, ?it/s]


  ✓ 0개 해결 | 남은 결측치: 0개

> Step 2: 동일 Publisher
  ✓ 0개 해결 | 남은 결측치: 0개

> Step 3: Book Title 키워드 매칭


  처리 중: 0it [00:00, ?it/s]

  ✓ 0개 해결 | 남은 결측치: 0개

> Step 4: Sentence-BERT 클러스터링 (목표: 650개)
** 모델 로딩 시작


** 임베딩 생성 시작
** 클러스터링 시작
  ✓ 0개 해결 | 남은 결측치: 0개

> Category 결측치 처리 완료
Step 1 (저자+연도): 0개
Step 2 (Publisher): 0개
Step 3 (키워드): 0개
Step 4 (클러스터링): 0개
Step 5 (unknown): 0개
총 해결: 0개 (nan%)

** Book Author: 결측치 1건 → 'unknown'으로 대체
> Books 정제 완료

> Ratings 데이터 정제 시작
> Ratings 정제 완료

>>> 정제된 데이터 저장 시작
** 저장 경로: /data/ephemeral/home/sojin/data/v4/
  - /data/ephemeral/home/sojin/data/v4/users.csv
  - /data/ephemeral/home/sojin/data/v4/books.csv
  - /data/ephemeral/home/sojin/data/v4/train_ratings.csv
  - /data/ephemeral/home/sojin/data/v4/test_ratings.csv

<<< 저장 완료! >>>

                    >>> 데이터 정제 결과 >>> 

[원본 데이터 크기]
  - Users: 68,092 rows
  - Books: 149,570 rows
  - Train: 306,795 rows
  - Test: 76,699 rows

[Age 정제]
- 결측치 대체: 27,833건

[Location 정제]
- Country 복원: 586건
- State 복원: 1140건
- Country 최빈값: usa

[Books 정제]
- 출판연도 이상치: 3건
- 사용된 중앙 연도: 1996
- Language 결측치: 67,227건
- Category 결측치: 0건
- Author 결측치: 1건

[Advanced Category 처리 상세]
- 원본 결측치: 0건
- Step 1 (저자+연도): 0건
- Step 2 (Publisher)